# Notebook to Fit Spectra

## Load Libraries

In [1]:
from multiprocessing import Pool, cpu_count

In [2]:
# Find out how many cpus you have
num_cpus = cpu_count()
print('NCPU available = ',num_cpus)


NCPU available =  8


Make a dummy function that we can send to the workers in the pool. 
Returns a sum of val with itself, however that is defined for val

In [3]:
def printer(val):
    step=0
    #Something that takes actual "computing time", so one can 
    #check in system monitor that all cpus are actually working
    for i in range(10**6): 
        step+=1
    print('The input is: ', val)
    return val+val


In [4]:
# Make the pool object
p = Pool(processes=num_cpus)
# Make some input to map out to the workers
nums_to_print = range(num_cpus)
M = [[i*10+j for j in range(num_cpus)] for i in range(num_cpus)]


The input is: The input is:   [0, 1, 2, 3, 4, 5, 6, 7][10, 11, 12, 13, 14, 15, 16, 17]
The input is: The input is: 
 The input is:  [30, 31, 32, 33, 34, 35, 36, 37]The input is: [40, 41, 42, 43, 44, 45, 46, 47] 

[50, 51, 52, 53, 54, 55, 56, 57] 
[20, 21, 22, 23, 24, 25, 26, 27]
The input is:  [70, 71, 72, 73, 74, 75, 76, 77]
The input is:  [60, 61, 62, 63, 64, 65, 66, 67]


### EXAMPLE 1
Input is a matrix, each thread is assigned a column

In [5]:
#Synchronous exectuion, later code inexecutable before all workers ready
#results = p.map(printer, nums_to_print)
results = p.map(printer, M)

#The asynchronous version has some further features.
#results = p.map_async(printer, nums_to_print)
#print('Results ready = ',results.ready())  #True if all workers finished
#print(results.successful())  #True if all workers successful
# To actually retrieve the result from the async result object, use get
#results = results.get()

print('\nEx.1 results = ')
print(results,'\n')



Ex.1 results = 
[[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7], [10, 11, 12, 13, 14, 15, 16, 17, 10, 11, 12, 13, 14, 15, 16, 17], [20, 21, 22, 23, 24, 25, 26, 27, 20, 21, 22, 23, 24, 25, 26, 27], [30, 31, 32, 33, 34, 35, 36, 37, 30, 31, 32, 33, 34, 35, 36, 37], [40, 41, 42, 43, 44, 45, 46, 47, 40, 41, 42, 43, 44, 45, 46, 47], [50, 51, 52, 53, 54, 55, 56, 57, 50, 51, 52, 53, 54, 55, 56, 57], [60, 61, 62, 63, 64, 65, 66, 67, 60, 61, 62, 63, 64, 65, 66, 67], [70, 71, 72, 73, 74, 75, 76, 77, 70, 71, 72, 73, 74, 75, 76, 77]] 



### EXAMPLE 2
Each thread is assigned the same matrix

In [6]:
usecpus=3 #Don't use all cpus, simple output is easy to print

p2 = Pool(processes=usecpus)
M2 = [[i*10+j for j in range(usecpus)] for i in range(usecpus)]


#Synchronous exectuion, later code inexecutable before all workers ready
results2 = p2.map(printer, [M2 for _ in range(usecpus)])

for i in range(len(results2)):
    print('\nEx.2 results[',i,'] = ')
    print(results2[i],'\n')

print('Original input object remains unmodified:')
print(M2)


The input is: The input is: The input is:   [[0, 1, 2], [10, 11, 12], [20, 21, 22]] [[0, 1, 2], [10, 11, 12], [20, 21, 22]][[0, 1, 2], [10, 11, 12], [20, 21, 22]]



Ex.2 results[ 0 ] = 
[[0, 1, 2], [10, 11, 12], [20, 21, 22], [0, 1, 2], [10, 11, 12], [20, 21, 22]] 


Ex.2 results[ 1 ] = 
[[0, 1, 2], [10, 11, 12], [20, 21, 22], [0, 1, 2], [10, 11, 12], [20, 21, 22]] 


Ex.2 results[ 2 ] = 
[[0, 1, 2], [10, 11, 12], [20, 21, 22], [0, 1, 2], [10, 11, 12], [20, 21, 22]] 

Original input object remains unmodified:
[[0, 1, 2], [10, 11, 12], [20, 21, 22]]


### Example 3: too many processes vs Ncpu

Let's run a function printing a single number, going through a large vector either linearly or by assigning each element a thread of it's own. The smart way to do this would be to divide the vector into Ncpu parts and run over them in parallel with a simple algorithm, but let's see how much we lose with a naive too-many-threads approach.

In [ ]:
#%%time
#
## Simple linear printing
#Nhuge=50
#results3a=[printer(i) for i in range(Nhuge)]
#
#### The below implementation would actually crash due to "too many open files" 
#### = presumably trying to instantiate too many threads
#
## Make the pool object
#p3 = Pool(processes=Nhuge)
## Make some input to map out to the workers
#lotsOfNumbers = range(Nhuge)
#results2 = p3.map(printer, lotsOfNumbers)